In [1]:
import coverage_v1 as cov
import geopandas as gpd
import plotly.express as px

In [2]:
tles = cov.gen_sats(sat_nos=[39084, 49260])
times = cov.gen_times(start_yr=2021, start_mo=11, start_day=20, days=1, step_min=1)
df1 = cov.prop_orbits(tles, times)
df1 = cov.add_bearing(df1)
dt_df = df1.set_index("time")


Satellite(s) Loaded from TLE:
[<EarthSatellite LANDSAT 8 catalog #39084 epoch 2021-11-28 03:42:11 UTC>]
[<EarthSatellite LANDSAT 9 catalog #49260 epoch 2021-11-28 16:08:40 UTC>]
Propogation time: 
 2021-11-20 00:00:00+00:00 
to 
 2021-11-20 23:59:00+00:00
Adding LANDSAT 8
Adding LANDSAT 9


In [3]:
# fig1 = px.scatter_geo(dt_df, lon="lon", lat="lat", color="satellite", projection="orthographic")
# fig1.show()

In [4]:
df1

,time,satellite,lat,lon,altitude,bearing,sunlit
0,2021-11-20 00:00:00+00:00,LANDSAT 8,81.730943,-125.977520,715.100005,261.268799,True
1,2021-11-20 00:01:00+00:00,LANDSAT 8,80.466116,-148.385907,714.980343,239.612943,True
2,2021-11-20 00:02:00+00:00,LANDSAT 8,78.186791,-164.003053,714.752163,224.746653,True
3,2021-11-20 00:03:00+00:00,LANDSAT 8,75.358801,-174.255330,714.419564,215.243090,True
4,2021-11-20 00:04:00+00:00,LANDSAT 8,72.241681,178.797842,713.988716,209.042396,True
...,...,...,...,...,...,...,...
2875,2021-11-20 23:55:00+00:00,LANDSAT 9,-12.094112,-23.876822,698.360706,347.859960,False
2876,2021-11-20 23:56:00+00:00,LANDSAT 9,-8.465317,-24.665272,697.628862,347.928867,False
2877,2021-11-20 23:57:00+00:00,LANDSAT 9,-4.834619,-25.443821,697.026929,347.978101,False
2878,2021-11-20 23:58:00+00:00,LANDSAT 9,-1.202680,-26.216847,696.560658,348.008915,False


In [5]:
inst = cov.gen_instrument(name="instrument", fl=100, pitch=0.017, h_pix=1024*3, v_pix=768, mm=True)
inst

{'name': 'instrument',
 'fl': 100,
 'pitch': 0.017,
 'h_pix': 3072,
 'v_pix': 768,
 'mm': True,
 'half_diag_deg': 15.421522038478695,
 'az1': 75.96375653207353,
 'az2': 284.03624346792645}

In [6]:
df1

,time,satellite,lat,lon,altitude,bearing,sunlit
0,2021-11-20 00:00:00+00:00,LANDSAT 8,81.730943,-125.977520,715.100005,261.268799,True
1,2021-11-20 00:01:00+00:00,LANDSAT 8,80.466116,-148.385907,714.980343,239.612943,True
2,2021-11-20 00:02:00+00:00,LANDSAT 8,78.186791,-164.003053,714.752163,224.746653,True
3,2021-11-20 00:03:00+00:00,LANDSAT 8,75.358801,-174.255330,714.419564,215.243090,True
4,2021-11-20 00:04:00+00:00,LANDSAT 8,72.241681,178.797842,713.988716,209.042396,True
...,...,...,...,...,...,...,...
2875,2021-11-20 23:55:00+00:00,LANDSAT 9,-12.094112,-23.876822,698.360706,347.859960,False
2876,2021-11-20 23:56:00+00:00,LANDSAT 9,-8.465317,-24.665272,697.628862,347.928867,False
2877,2021-11-20 23:57:00+00:00,LANDSAT 9,-4.834619,-25.443821,697.026929,347.978101,False
2878,2021-11-20 23:58:00+00:00,LANDSAT 9,-1.202680,-26.216847,696.560658,348.008915,False


In [7]:
# df = df1[df1.satellite == "LANDSAT 8"]
df = df1
df.loc[0]

time         2021-11-20 00:00:00+00:00
satellite                    LANDSAT 8
lat                          81.730943
lon                         -125.97752
altitude                    715.100005
bearing                     261.268799
sunlit                            True
Name: 0, dtype: object

In [8]:
df1 = df1.dropna()

In [9]:
df, polys = cov.gen_los_offsets(df1, inst)

In [10]:
cmap = px.colors.qualitative.Plotly
sats = polys.satellite.unique()
color_key = dict(zip(sats, cmap))

color_key


polys['color'] = polys.apply(
    lambda row: color_key[row.satellite], axis=1)



In [11]:
polys = polys.to_crs("ESRI:54009")
polys = polys[polys.area < polys.area.quantile(0.98)]
polys

,geometry,satellite,color
0,"POLYGON ((-3305587.463 8733490.113, -4155933.1...",LANDSAT 8,#636EFA
1,"POLYGON ((-4561627.869 8632472.545, -5140789.5...",LANDSAT 8,#636EFA
2,"POLYGON ((-5981423.292 8459704.700, -6303210.8...",LANDSAT 8,#636EFA
5,"POLYGON ((8789035.479 7709397.536, 9616048.543...",LANDSAT 8,#636EFA
6,"POLYGON ((9478720.011 7409365.423, 10219541.96...",LANDSAT 8,#636EFA
...,...,...,...
2873,"POLYGON ((-2105502.717 -1839174.943, -2435964....",LANDSAT 9,#EF553B
2874,"POLYGON ((-2206684.767 -1395929.505, -2535983....",LANDSAT 9,#EF553B
2875,"POLYGON ((-2301482.540 -950128.416, -2630236.1...",LANDSAT 9,#EF553B
2876,"POLYGON ((-2389756.749 -502550.376, -2718576.0...",LANDSAT 9,#EF553B


In [12]:
polys.explore(color="color", tooltip="satellite")
# for tomorrow... https://towardsdatascience.com/around-the-world-in-80-lines-crossing-the-antimeridian-with-python-and-shapely-c87c9b6e1513

In [13]:
df.reset_index()

,index,time,satellite,lat,lon,altitude,bearing,sunlit
0,0,2021-11-20 00:00:00+00:00,LANDSAT 8,81.730943,-125.977520,715.100005,261.268799,True
1,1,2021-11-20 00:01:00+00:00,LANDSAT 8,80.466116,-148.385907,714.980343,239.612943,True
2,2,2021-11-20 00:02:00+00:00,LANDSAT 8,78.186791,-164.003053,714.752163,224.746653,True
3,3,2021-11-20 00:03:00+00:00,LANDSAT 8,75.358801,-174.255330,714.419564,215.243090,True
4,4,2021-11-20 00:04:00+00:00,LANDSAT 8,72.241681,178.797842,713.988716,209.042396,True
...,...,...,...,...,...,...,...,...
2873,2874,2021-11-20 23:54:00+00:00,LANDSAT 9,-15.720334,-23.073771,699.214467,347.769332,False
2874,2875,2021-11-20 23:55:00+00:00,LANDSAT 9,-12.094112,-23.876822,698.360706,347.859960,False
2875,2876,2021-11-20 23:56:00+00:00,LANDSAT 9,-8.465317,-24.665272,697.628862,347.928867,False
2876,2877,2021-11-20 23:57:00+00:00,LANDSAT 9,-4.834619,-25.443821,697.026929,347.978101,False
